In [1]:
import pandas as pd

In [3]:
pop_com = pd.read_csv('population_communes.csv')
dep_surface = pd.read_csv('surface_departements.csv')

In [4]:
pop_com.head()

,Code département,Code canton,Code arrondissement,Code région,Nom de la commune,Code commune,Nom de la région,variable,value
0,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population totale,184
1,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population municipale,177
2,55,10.0,2,44,Abainville,1,Grand Est,Population totale,310
3,55,10.0,2,44,Abainville,1,Grand Est,Population municipale,305
4,60,11.0,1,32,Abancourt,1,Hauts-de-France,Population totale,658


In [5]:
dep_surface.head()

,code_insee,nom,surf_km2
0,974,La Réunion,2505.0
1,11,Aude,6343.0
2,43,Haute-Loire,5003.0
3,13,Bouches-du-Rhône,5247.0
4,47,Lot-et-Garonne,5385.0


In [100]:
df_merge = pop_com.merge(how='inner', right=dep_surface, left_on='Code département', right_on='code_insee')
df_merge.head()

,Code département,Code canton,Code arrondissement,Code région,Nom de la commune,Code commune,Nom de la région,variable,value,code_insee,nom,surf_km2
0,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population totale,184,64,Pyrénées-Atlantiques,7683.0
1,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population municipale,177,64,Pyrénées-Atlantiques,7683.0
2,64,9.0,3,75,Abidos,3,Nouvelle-Aquitaine,Population totale,239,64,Pyrénées-Atlantiques,7683.0
3,64,9.0,3,75,Abidos,3,Nouvelle-Aquitaine,Population municipale,232,64,Pyrénées-Atlantiques,7683.0
4,64,16.0,2,75,Abitain,4,Nouvelle-Aquitaine,Population municipale,97,64,Pyrénées-Atlantiques,7683.0


In [101]:
sum_pop_mun = df_merge.loc[df_merge.variable == 'Population municipale', ['Code département', 'value']]\
                      .groupby(['Code département']).agg({'value': sum})\
                      .rename(columns={'value': 'population municipale'})

sum_pop_tot = df_merge.loc[df_merge.variable == 'Population totale', ['Code département', 'value']]\
                      .groupby(['Code département']).agg({'value': sum})\
                      .rename(columns={'value': 'population totale'})

df_result = sum_pop_mun.merge(sum_pop_tot, right_index=True, left_index=True)
df_result.head()

,population municipale,population totale
Code département,,
01,638425,655171
02,536136,549587
03,339384,349336
04,162565,167331
05,141107,146148


In [102]:
df_result['pourcent_pop_mun'] = df_result['population municipale']/df_result['population totale']
df_result.head()

,population municipale,population totale,pourcent_pop_mun
Code département,,,
01,638425,655171,0.974440
02,536136,549587,0.975525
03,339384,349336,0.971512
04,162565,167331,0.971518
05,141107,146148,0.965508


In [98]:
# Transform préserve le nombre de colonne initiale
pop_mun = df_result.groupby('Code région').transform('sum')['population municipale']

In [99]:
pop_mun

0      7916889
1      6006870
2      7916889
3      5021928
4      5021928
        ...   
95    12117132
96      394110
97      376480
98      269352
99      852924
Name: population municipale, Length: 100, dtype: object